In [ ]:
import torch
import numpy as np
import cv2
import time
import datetime
import pandas as pd
from PIL import Image
from yolov5.models.experimental import attempt_load
import warnings
warnings.filterwarnings(action = 'ignore')

from convert_data import collect_data, collect_coordinates
from class_mapping import yolo_class_mapping, fire_class_mapping, animals_class_mapping, obstacle_class_mapping
from detection import category
from perform_object import perform_object
from yolo_to_server import send_server

### Version check

In [ ]:
print('opencv version: ', cv2.__version__)

### Set Trained Models & Camera setting

In [ ]:
camera_name = 'KFQ StarValley'
device = torch.device('cpu')

# Load trained YOLOv5 model weights
yolo_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/yolov5m.pt'
fire_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/fire_smoke.pt'
animals_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/animals.pt'
obstacle_weights_path = '/Users/bongeungu/Desktop/kfq/KFQ_TEAM01/yolo/models/obstacle.pt'

yolo_model = attempt_load(yolo_weights_path, device=device)
fire_model = attempt_load(fire_weights_path, device=device)
animals_model = attempt_load(animals_weights_path, device=device)
obstacle_model = attempt_load(obstacle_weights_path, device=device)

### Save coordinates for road boundary detection

In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)

columns = ['X1', 'Y1', 'X2', 'Y2', 'Class_Name']
road_coordinates = pd.DataFrame(columns=columns)

for i in range(200):
    ret, frame = cap.read()
    
    image = Image.fromarray(frame)
    image = image.resize((640,640))
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)
    
    results = perform_object(image_tensor, 0.65, yolo_model)
    road_coordinates = collect_coordinates(results, frame, yolo_class_mapping, road_coordinates)
    
road_coordinates = road_coordinates.sort_values(by = ['X1', 'Y1'])
print(road_coordinates)
print(f'coodrinates count: {len(road_coordinates)}')

### Object Detection using webcam

In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    captured_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # 상황 발생 시간 기록
    print(captured_time)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit the loop
        break
    
    # reset dataframe
    columns = ['Time', 'Class_Name', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'action_detection', 'action_category', 'event_type']
    results_df = pd.DataFrame(columns=columns)
    
    # Preprocess the frame
    image = Image.fromarray(frame)
    image = image.resize((640, 640))  # Resize the image to the model's input size
    image_tensor = torch.from_numpy(np.array(image)).float().permute(2, 0, 1) / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    # Perform object detection
    results = perform_object(image_tensor, 0.65, yolo_model)
    results_df, road_coordinates = collect_data(results, frame, yolo_class_mapping, results_df, captured_time, road_coordinates)
    #print(results_df)
    
    results = perform_object(image_tensor, 0.3, fire_model)
    results_df, road_coordinates = collect_data(results, frame, fire_class_mapping, results_df, captured_time, road_coordinates)
    
    results = perform_object(image_tensor, 0.65, animals_model)
    results_df, road_coordinates = collect_data(results, frame, animals_class_mapping, results_df, captured_time, road_coordinates)
    
    results = perform_object(image_tensor, 0.4, obstacle_model)
    results_df, road_coordinates = collect_data(results, frame, obstacle_class_mapping, results_df, captured_time, road_coordinates)
    
    #print(road_coordinates)
    
    results_df = category(results_df, road_coordinates)

    # Display the frame with bounding boxes and labels
    cv2.imshow(camera_name, frame)
    
    send_server(results_df, camera_name, frame)
    
    time.sleep(2) # 2초 간격으로 사진 캡쳐(30 frames per 1 miniute)

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()